# Vision Edge-AI와 3D Depth Camera

3D Depth Camera는 공간의 깊이 정보를 캡처하여 3차원 데이터로 변환하는 장치입니다. 이러한 카메라는 다양한 응용 분야에서 사용되며, 컴퓨터 비전, 로봇 공학, 증강현실(AR), 가상현실(VR), 의료 이미지 처리 등에서 중요합니다. 본 강의에서는 3D Depth Camera의 원리, 기술, 응용 사례, 그리고 DeepStream을 활용할 수 있는 방법에 대해 설명합니다.

## 03-2. 3D Depth Camera

### 3D Depth Camera의 정의
3D Depth Camera는 객체나 장면의 깊이 정보를 캡처할 수 있는 장치로, 카메라가 각 픽셀의 거리 값을 캡처하여 3차원 공간에서 물체를 인식하거나 측정할 수 있도록 합니다.

### 3D Depth Camera의 원리
1. **스테레오 비전 (Stereo Vision)**:
    * 두 개의 카메라로 물체를 촬영한 이미지를 비교하여 깊이를 계산합니다.
    * 사람의 시각 시스템과 유사한 방식입니다.

2. **구조광 (Structured Light)**:
    * 특정 패턴의 빛을 물체에 투사하고, 왜곡된 패턴을 분석하여 깊이를 측정합니다.
    * 대표적인 예: Microsoft Kinect

3. **비행시간 (Time of Flight, ToF)**:
    * 빛이 물체에 반사되어 돌아오는 시간을 측정하여 깊이를 계산합니다.
    * 정확하고 빠른 깊이 측정이 가능합니다.

4. **LiDAR (Light Detection and Ranging)**:
    * 레이저를 사용해 주변 환경을 스캔하여 깊이를 측정합니다.
    * 자율주행차에 주로 사용됩니다.

### 응용 분야
1. **산업 및 제조**:
    * 로봇 비전: 로봇의 정확한 동작과 위치 결정을 지원.
    * 품질 검사: 제품의 결함 및 크기 측정.
2. **의료**:
    * 3D 스캔: 인체를 스캔하여 맞춤형 의료 장비 제작.
    * 수술 보조: 정확한 수술 계획을 위한 3D 모델 생성.
3. **엔터테인먼트**:
    * 증강현실/가상현실(AR/VR): 몰입형 환경 구축.
    * 게임: 사용자 동작을 추적하여 인터랙션 제공.
4. **도매(Retail)**:
    * 고객 행동 분석: 매장 내 고객의 동선을 추적하고 행동을 분석.
    * 재고 관리: 제품 크기 및 위치를 자동으로 스캔하여 재고를 효율적으로 관리.
    * 무인 매장: 3D 데이터를 활용해 고객과 제품의 상호작용을 자동화.

## 03-3. OAK-D 모듈
- **OAK-D 모듈 소개**: OAK-D(OpenCV AI Kit Depth)는 AI 및 컴퓨터 비전 기능을 갖춘 카메라 모듈로, 실시간으로 깊이 정보를 제공하며 다양한 응용 분야에 적합합니다.

- **주요 특징**:
  1. **내장형 AI**: Myriad X VPU를 탑재하여 온보드 AI 처리 가능.
  2. **스테레오 비전**: 두 개의 모노 카메라로 깊이 정보 계산.
  3. **컬러 카메라**: 고해상도 컬러 카메라로 세부 정보를 캡처.
  4. **플러그 앤 플레이**: USB를 통해 쉽게 연결 및 사용 가능.
  5. **다양한 SDK 지원**: DepthAI 및 OpenCV와 같은 SDK 지원.

- **응용 사례**:
  - 로봇 내비게이션
  - 객체 추적 및 감지
  - 증강 현실 및 가상 현실 애플리케이션

### OAK-D PRO의 자체 기능과 기술
- **IMU 센서**: 내장된 IMU 센서로 자세와 움직임을 추적할 수 있다.
- **AI 프로세싱**: 내장된 AI 칩으로 머신러닝 모델을 로컬에서 실행 가능.
- **다중 카메라**: 컬러 카메라와 두 개의 모노 카메라를 사용하여 스테레오 비전 지원.
- **ROS 지원**: 로봇 운영 체제(ROS)와의 통합 용이.

---

# 간단한 예제 실습

## Jetson Orin Nano에서 Camera module setting (Connection)

### 준비물
- Jetson Orin Nano
- OAK-D PRO Camera
- USB-C 케이블

### 필수 라이브러리 설치
라이브러리 설치
```bash
# 설치되어있는 OpenCV 삭제
$ pip uninstall opencv-python

# 저장소 추가
$ sudo add-apt-repository universe
$ sudo apt update

# 필수 라이브러리 설치
$ sudo apt install -y build-essential cmake git pkg-config libgtk-3-dev libavcodec-dev libavformat-dev libswscale-dev libv4l-dev libxvidcore-dev libx264-dev libjpeg-dev libpng-dev libtiff-dev gfortran openexr libatlas-base-dev python3-dev python3-numpy libtbb2 libtbb-dev libdc1394-dev

# OpenCV 설치
pip install opencv-python
```

OAK-D 관련 드라이버 설치
```bash
$ wget -qO- https://docs.luxonis.com/install_dependencies.sh | bash
$ pip install depthai --upgrade
```
* 드라이버 설치 이후 반드시 OAK-D 모듈 연결 USB를 뺐다가 다시 꽂기

OpenCV 방식

In [2]:
import cv2
import depthai as dai

# Create pipeline
pipeline = dai.Pipeline()

# Create ColorCamera node
cam = pipeline.create(dai.node.ColorCamera)
cam.setBoardSocket(dai.CameraBoardSocket.CAM_A)  # CAM_A로 수정
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam.setVideoSize(1920, 1080)

# Create XLinkOut node
xout = pipeline.create(dai.node.XLinkOut)
xout.setStreamName("video")

# Connect camera to output
cam.video.link(xout.input)

# Start the pipeline
with dai.Device(pipeline) as device:
    print("Pipeline started")
    video = device.getOutputQueue(name="video", maxSize=4, blocking=False)
    
    while True:
        # Get frame
        frame = video.get().getCvFrame()
        
        # Show frame
        cv2.imshow("OAK-D Video Feed", frame)
        
        # Exit on 'q' key
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()

Pipeline started


Matplotlib 방식

In [ ]:
import cv2
import depthai as dai
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Create pipeline
pipeline = dai.Pipeline()

# Create ColorCamera node
cam = pipeline.create(dai.node.ColorCamera)
cam.setBoardSocket(dai.CameraBoardSocket.CAM_A)
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam.setVideoSize(1920, 1080)

# Create XLinkOut node
xout = pipeline.create(dai.node.XLinkOut)
xout.setStreamName("video")

# Connect camera to output
cam.video.link(xout.input)

# Start the pipeline
with dai.Device(pipeline) as device:
    print("Pipeline started")
    video = device.getOutputQueue(name="video", maxSize=4, blocking=False)
    
    while True:
        # Get frame
        frame = video.get().getCvFrame()
        
        # Convert to RGB for Matplotlib
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Display frame in Jupyter Notebook
        clear_output(wait=True)
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.show()


KeyboardInterrupt: 

## OAK-D와 DeepStream을 결합한 ObjectDetection

### 준비물
* Jetson
* DeepStream 설치
* OAK-D 카메라 모듈

### 실행 준비
폴더 만들기
```bash
$ mkdir jetson-oakd
```

DeepStream 파이프라인 파일 작성(deepstream-oakd.c)
```c
#include <gst/gst.h>
#include <glib.h>
#include <stdio.h>
#include <cuda_runtime_api.h>
#include "gstnvdsmeta.h"
#include "nvds_yml_parser.h"
#include <depthai/depthai.hpp>

#define MAX_DISPLAY_LEN 64

gint frame_number = 0;
gchar pgie_classes_str[4][32] = {"Vehicle", "TwoWheeler", "Person", "Roadsign"};

typedef struct
{
    dai::Pipeline pipeline;
    std::shared_ptr<dai::Device> device;
    std::shared_ptr<dai::DataOutputQueue> rgbQueue;
} OAKDSource;

// OAK-D 초기화
OAKDSource init_oakd()
{
    OAKDSource oakdSource;
    oakdSource.pipeline = dai::Pipeline();

    // ColorCamera 노드 생성
    auto camRgb = oakdSource.pipeline.create<dai::node::ColorCamera>();
    camRgb->setResolution(dai::ColorCameraProperties::SensorResolution::THE_1080_P);
    camRgb->setInterleaved(false);
    camRgb->setColorOrder(dai::ColorCameraProperties::ColorOrder::BGR);

    // XLinkOut 노드 생성
    auto xoutRgb = oakdSource.pipeline.create<dai::node::XLinkOut>();
    xoutRgb->setStreamName("rgb");
    camRgb->video.link(xoutRgb->input);

    // OAK-D 장치 연결
    oakdSource.device = std::make_shared<dai::Device>(oakdSource.pipeline);

    // RGB 출력 큐 생성
    oakdSource.rgbQueue = oakdSource.device->getOutputQueue("rgb", 8, false);

    return oakdSource;
}

// appsrc 데이터 푸시 콜백
GstFlowReturn push_frame_to_gstreamer(GstElement *appsrc, guint size, gpointer user_data)
{
    OAKDSource *oakdSource = (OAKDSource *)user_data;
    auto imgFrame = oakdSource->rgbQueue->get<dai::ImgFrame>();
    if (!imgFrame)
    {
        return GST_FLOW_EOS;
    }

    // RGB 데이터를 GStreamer 버퍼로 변환
    GstBuffer *buffer = gst_buffer_new_wrapped(imgFrame->getData().data(), imgFrame->getData().size());

    // 버퍼 푸시
    GstFlowReturn ret;
    g_signal_emit_by_name(appsrc, "push-buffer", buffer, &ret);
    gst_buffer_unref(buffer);

    return ret;
}

int main(int argc, char *argv[])
{
    GstElement *pipeline, *appsrc, *h264parser, *decoder, *sink;
    GstBus *bus;
    GMainLoop *loop;

    // GStreamer 초기화
    gst_init(&argc, &argv);

    // OAK-D 초기화
    OAKDSource oakdSource = init_oakd();

    // GStreamer 파이프라인 생성
    pipeline = gst_pipeline_new("oakd-pipeline");

    // appsrc 생성
    appsrc = gst_element_factory_make("appsrc", "oakd-appsrc");
    g_object_set(G_OBJECT(appsrc), "caps", gst_caps_from_string("video/x-raw,format=BGR,width=1920,height=1080,framerate=30/1"), NULL);
    g_signal_connect(appsrc, "need-data", G_CALLBACK(push_frame_to_gstreamer), &oakdSource);

    // GStreamer 요소 생성
    h264parser = gst_element_factory_make("h264parse", "h264-parser");
    decoder = gst_element_factory_make("nvv4l2decoder", "nvv4l2-decoder");
    sink = gst_element_factory_make("nveglglessink", "video-output");

    if (!pipeline || !appsrc || !h264parser || !decoder || !sink)
    {
        g_printerr("Failed to create GStreamer elements.\n");
        return -1;
    }

    // 파이프라인에 요소 추가
    gst_bin_add_many(GST_BIN(pipeline), appsrc, h264parser, decoder, sink, NULL);

    // 요소 연결
    if (!gst_element_link_many(appsrc, h264parser, decoder, sink, NULL))
    {
        g_printerr("Failed to link elements.\n");
        return -1;
    }

    // 파이프라인 실행
    gst_element_set_state(pipeline, GST_STATE_PLAYING);

    // 이벤트 루프 시작
    loop = g_main_loop_new(NULL, FALSE);
    g_main_loop_run(loop);

    // 종료 처리
    gst_element_set_state(pipeline, GST_STATE_NULL);
    gst_object_unref(pipeline);
    g_main_loop_unref(loop);

    return 0;
}

```